In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# import the required libraries
from torchvision import datasets,transforms
from torch.utils.data import Dataset, DataLoader
import torch
import os
from PIL import Image
import numpy as np
import pandas as pd

In [3]:
# define device to use the gpu
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [4]:
# image directory
image_dir='/kaggle/input/mydataset/images'


In [5]:
# ocnvert the image directory to image list containing the image name example : image.png
image_filenames=os.listdir(image_dir)

In [7]:
# define the list of all images
images = [Image.open(os.path.join(image_dir, filename)) for filename in image_filenames]


In [8]:
# number of data points or images in images list
len(images)

622

In [ ]:
# images = torch.stack([preprocess(image) for image in images])


In [9]:
# ocnvert the annotation of image into pandas's dataframe
df=pd.read_csv('/kaggle/input/label-csv/role_challenge_dataset_ground_truth.csv')

In [10]:
# first five example of corresponding image name labels
df.head()

,image_name,ofd_1_x,ofd_1_y,ofd_2_x,ofd_2_y,bpd_1_x,bpd_1_y,bpd_2_x,bpd_2_y
0,000_HC.png,361,12,339,530,481,16,664,318
1,001_HC.png,441,331,368,308,297,247,534,142
2,002_HC.png,318,374,154,406,481,158,558,215
3,003_HC.png,424,105,407,462,305,349,547,363
4,004_HC.png,300,277,611,534,53,452,494,308


In [ ]:
for _,row in df.iterrows():
    pass
   

In [11]:
# number of columns contain the annotation file
df.columns

Index(['image_name', 'ofd_1_x', 'ofd_1_y', 'ofd_2_x', 'ofd_2_y', 'bpd_1_x',
       'bpd_1_y', 'bpd_2_x', 'bpd_2_y'],
      dtype='object')

In [12]:
# map the image name to its coressponding lebels
image_landmark_map = {row['image_name']: [row['ofd_1_x'], row['ofd_1_y'],row['ofd_2_x'],row['ofd_2_y'],row['bpd_1_x'],row['bpd_1_y'],row['bpd_2_x'],row['bpd_2_y']] for _, row in df.iterrows()}


In [13]:
# combine the images and its labels
formatted_data = [(image, image_landmark_map[filename]) for image, filename in zip(images, image_filenames)]


In [15]:
# example of image and its coresponding cordinates
print(f"image: {formatted_data[0][0]}\n")
print(f"label: {formatted_data[0][1]}")

image: <PIL.PngImagePlugin.PngImageFile image mode=L size=800x540 at 0x787CDEE3CE80>

label: [371, 70, 309, 451, 115, 273, 642, 393]


In [16]:
# convert the label data frame into numpy array
label=(np.array(df))

In [17]:
label

array([['000_HC.png', 361, 12, ..., 16, 664, 318],
       ['001_HC.png', 441, 331, ..., 247, 534, 142],
       ['002_HC.png', 318, 374, ..., 158, 558, 215],
       ...,
       ['499_2HC.png', 432, 105, ..., 284, 647, 296],
       ['499_HC.png', 356, 119, ..., 347, 608, 272],
       ['500_HC.png', 429, 70, ..., 258, 652, 213]], dtype=object)

In [18]:
# Define transformations for augmentation of image
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [19]:
# difine a list of containing the label
landmarks=[label for _,label in formatted_data]

In [20]:
len(landmarks[0])

8

In [21]:
len(landmarks)

622

In [ ]:
transform=transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# landmarks

In [22]:
# convert the landmarks into tensor format 
landmarks_tensor = torch.tensor(landmarks,dtype=torch.float32)


In [23]:
landmarks_tensor[0]

tensor([371.,  70., 309., 451., 115., 273., 642., 393.])

In [24]:
landmarks_tensor.shape

torch.Size([622, 8])

In [25]:
# Dataloader to wrap the image and label into batches
from torch.utils.data import Dataset, DataLoader

In [ ]:
# # define the second model
# class model_2(nn.Module):
#     def __init__(self,):
#         super().__init__()

In [26]:
# convert the image into the tensor format and stack them
images_tensor=torch.stack([preprocess(image) for image,_ in formatted_data])

In [27]:
# define dataloader for image and landmark 
image_dataloader=DataLoader(images_tensor,batch_size=16,shuffle=False)
landmarks_dataloader=DataLoader(landmarks_tensor,batch_size=16,shuffle=False)

In [28]:
import torchvision.models as models

In [29]:
pretrained_resnet = models.resnet18(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s] 


In [30]:
import torch.nn as nn

In [31]:
# change image channel in pretrained network for our image dataset
pretrained_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)


In [32]:
# add the 8 neuron for our task
pretrained_resnet.fc = nn.Linear(pretrained_resnet.fc.in_features, 8)

In [ ]:
def image_to_tensor():
    pass
    

In [ ]:
# def dataLoader(images,landmarks):
#     images_tensor = torch.stack([preprocess(image)for image in images])
#     image_dataloader=DataLoader(images_tensor,batch_size=16,shuffle=False)
#     landmarks_tensor = torch.tensor(landmarks, dtype=torch.float32)
#     landmarks_dataloader=DataLoader(landmarks_tensor,batch_size=16,shuffle=False)
#     return image_dataloader,landmarks_dataloader
    

In [ ]:
import torch.nn as nn

In [40]:
class LandmarkDetectionModel(nn.Module):
    def __init__(self):
        super(LandmarkDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 8)  # Output: 8 coordinates for each landmark point

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [41]:
model=LandmarkDetectionModel().to(device)

In [42]:
output_1=model(images_tensor[1].to(device))

In [43]:
output_1

tensor([[ 0.0486,  0.0009, -0.0558, -0.0384, -0.0215, -0.0150, -0.0214, -0.0173]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [55]:

# define the model architecture 

class LandmarkDetectionModel_2(nn.Module):
    def __init__(self):
        super(LandmarkDetectionModel_2, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        
        # Define pooling layers
        self.pool = nn.MaxPool2d(2, 2)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(512 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 8)  # Output: 8 coordinates for each landmark point
       
        # Define dropout layer
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Apply convolutional and pooling layers
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.pool(nn.functional.relu(self.conv4(x)))
        x = self.pool(nn.functional.relu(self.conv5(x)))
        
        # Flatten the output for fully connected layers
        x = x.view(-1, 512 * 8 * 8)
        
        # Apply fully connected layers with dropout
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x


In [56]:
# intialize the model and shift it on device
model_2=LandmarkDetectionModel_2().to(device)

In [57]:
output_2=model_2(images_tensor[1].to(device))


In [58]:
output_2

tensor([[-0.0242, -0.0118,  0.0418, -0.0294, -0.0398, -0.0396,  0.0439,  0.0433]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [38]:
# define the loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, y_true, y_pred):
        mse = torch.mean((y_true - y_pred) ** 2)
        rmse = torch.sqrt(mse)
        return rmse

# Create an instance of the RMSE loss
criterion = RMSELoss()


In [44]:
# define loss function and optmizer
# criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.002)

In [45]:
images_tensor.shape

torch.Size([622, 1, 256, 256])

In [46]:
output=model(images_tensor[0].to(device))

In [47]:
output

tensor([[ 0.0491, -0.0023, -0.0577, -0.0494, -0.0127, -0.0145, -0.0134, -0.0152]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
#  for param in pretrained_resnet.parameters(): 
#         param.requires_grad = False


In [ ]:
# pretrained_resnet.to(device)

In [ ]:
model_2

In [ ]:
output= model_2(images_tensor[0].to(device))

In [ ]:
output

In [ ]:
images_tensor[0]

In [ ]:
output

In [ ]:
output.shape

In [ ]:
output= model(images_tensor[0].to(device))
output

In [ ]:
images_tensor[1]

In [ ]:
model

In [ ]:
# # Train the model
# num_epochs = 30
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images,landmarks in zip(image_dataloader,landmarks_dataloader):
#         images, landmarks = images.to(device), landmarks.to(device)
#         optimizer.zero_grad()
#         outputs=pretrained_resnet(images)
#         loss = criterion(outputs, landmarks)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(image_dataloader):.4f}')

# # Save the trained model
# torch.save(model.state_dict(), 'landmark_detection_model.pth')

In [59]:
optimizer=torch.optim.Adam(model_2.parameters(),lr=0.002)

In [60]:
# Train the model
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for images,landmarks in zip(image_dataloader,landmarks_dataloader):
        images, landmarks = images.to(device), landmarks.to(device)
        optimizer.zero_grad()
        outputs = model_2(images)
        loss = criterion(outputs, landmarks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(image_dataloader):.4f}')

# Save the trained model
torch.save(model_2.state_dict(), 'landmark_detection_model_2.pth')

Epoch [1/30], Loss: 210.9918
Epoch [2/30], Loss: 104.3621
Epoch [3/30], Loss: 99.7092
Epoch [4/30], Loss: 98.4530
Epoch [5/30], Loss: 96.0667
Epoch [6/30], Loss: 96.7302
Epoch [7/30], Loss: 92.8839
Epoch [8/30], Loss: 92.3827
Epoch [9/30], Loss: 92.7652
Epoch [10/30], Loss: 91.4994
Epoch [11/30], Loss: 92.2332
Epoch [12/30], Loss: 90.9742
Epoch [13/30], Loss: 90.4162
Epoch [14/30], Loss: 86.0955
Epoch [15/30], Loss: 82.6936
Epoch [16/30], Loss: 83.3227
Epoch [17/30], Loss: 81.5951
Epoch [18/30], Loss: 81.8773
Epoch [19/30], Loss: 79.7207
Epoch [20/30], Loss: 81.4272
Epoch [21/30], Loss: 81.4137
Epoch [22/30], Loss: 81.7069
Epoch [23/30], Loss: 80.8086
Epoch [24/30], Loss: 80.3042
Epoch [25/30], Loss: 77.1529
Epoch [26/30], Loss: 76.9025
Epoch [27/30], Loss: 77.6874
Epoch [28/30], Loss: 79.7108
Epoch [29/30], Loss: 76.0474
Epoch [30/30], Loss: 77.3233


In [61]:
output_2= model_2(images_tensor[1].to(device))
output_2

tensor([[389.4634, 134.6282, 388.3868, 448.7248, 171.1540, 260.8532, 619.5228,
         269.7719]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [64]:
landmarks_tensor[1]

tensor([381.,  96., 390., 462., 132., 288., 633., 280.])

In [48]:
# Train the model
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for images,landmarks in zip(image_dataloader,landmarks_dataloader):
        images, landmarks = images.to(device), landmarks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, landmarks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(image_dataloader):.4f}')

# Save the trained model
torch.save(model.state_dict(), 'landmark_detection_model.pth')

Epoch [1/30], Loss: 117.4398
Epoch [2/30], Loss: 86.1935
Epoch [3/30], Loss: 82.7150
Epoch [4/30], Loss: 86.1513
Epoch [5/30], Loss: 80.8872
Epoch [6/30], Loss: 78.3343
Epoch [7/30], Loss: 83.8302
Epoch [8/30], Loss: 68.9801
Epoch [9/30], Loss: 78.8590
Epoch [10/30], Loss: 71.4485
Epoch [11/30], Loss: 59.4436
Epoch [12/30], Loss: 55.6692
Epoch [13/30], Loss: 57.1080
Epoch [14/30], Loss: 52.6813
Epoch [15/30], Loss: 58.6369
Epoch [16/30], Loss: 58.8805
Epoch [17/30], Loss: 69.5424
Epoch [18/30], Loss: 71.5788
Epoch [19/30], Loss: 72.6321
Epoch [20/30], Loss: 62.7138
Epoch [21/30], Loss: 56.9316
Epoch [22/30], Loss: 51.2693
Epoch [23/30], Loss: 47.6080
Epoch [24/30], Loss: 52.2225
Epoch [25/30], Loss: 54.1456
Epoch [26/30], Loss: 30.6867
Epoch [27/30], Loss: 29.3048
Epoch [28/30], Loss: 45.7529
Epoch [29/30], Loss: 49.0254
Epoch [30/30], Loss: 41.8417


In [49]:
output= model(images_tensor[1].to(device))

In [50]:
output

tensor([[327.9156,  72.1777, 321.1572, 371.9760, 136.6906, 232.7710, 533.1948,
         226.3174]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [51]:
landmarks_tensor[1]

tensor([381.,  96., 390., 462., 132., 288., 633., 280.])